## Setup

###CUDA10 (run one time only)

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%cd /usr/local/

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!rm -rf cuda
!ln -s /usr/local/cuda-10.0 /usr/local/cuda

In [ ]:
!stat cuda

###Libraries (run one time only)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!conda --version
# If !conda --version returns no results, install conda with :
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch

In [ ]:
!pip install numpy==1.21.4
!pip install opencv-python==4.5.4.58
!pip install matplotlib==3.5.0
!pip intall pillow==8.4.0
!pip install pyyaml==6.0
!pip install tensorboard==2.7.0
!pip install tqdm==4.62.3
!pip install future==0.18.2
!pip instal imutils==0.5.4
!pip install -U pyyaml

##Code

###Setup (always run on restarting kernel)

In [ ]:
# If not already mounted    
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys

# You should rename your project folder as 'EPFL_ML_project_2'

%env PYTHONPATH="$/env/python:/content/drive/MyDrive/EPFL_ML_project_2"
%cd drive/MyDrive/EPFL_ML_project_2/
sys.path.append('/content/drive/MyDrive/EPFL_ML_project_2')

### Helpers

In [ ]:
import os
import torch
import numpy as np
import logging
import datetime
from utils.config import read_config, DotConfig
from utils.builders import *
from utils.trainval import train, predict
from torch.utils.tensorboard import SummaryWriter

SEED = 45
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def run(exp_name):
    # Set random number generator seed for numpy
    rng = np.random.RandomState(SEED)
    # Get command line arguments and configuration dictionary
    config_path = 'configs/' + exp_name + '.yaml'
    config = read_config(config_path)
    config = DotConfig(config)
    config.name = exp_name

    # Set file for logging
    log_filename = config.name + '.log'
    log_dir = config.log_dir_path + config.name
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    log_filename = log_dir + '/log.log'
    if os.path.exists(log_filename):
        now = datetime.datetime.now()
        log_filename = log_dir + '/log_' + str(now.minute) + '_' + str(now.second) + '.log'
    logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(levelname)s: %(message)s')
    logging.info(f'Configuration file used is <{config.name}>\n')

    # Check for cuda availability
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Device is {device}')

    # Build network according to config file and send it to device
    net = build_network(config)
    net.to(device=device)

    # Show network format for a 400x400x3 image input
    #summary(net, (3, 400, 400), 1)

    # Build dataset according to config file
    dataset = build_dataset(config)

    # Load pretrained VGG13
    if config.pretrain is not None:
        net = load_pretrain_model(net, config)
        logging.info(f'Loaded pretrained weights!\n')

    # Train network
    writer = SummaryWriter(log_dir=log_dir)
    train(net, dataset, config, writer, rng=rng, device=device)


def make_prediction(exp_name, model_checkpoint, save=False):
    args = {}
    args['save'] = save
    args = DotConfig(args)
     # Get command line arguments and configuration dictionary
    config_path = 'configs/' + exp_name + '.yaml'
    config = read_config(config_path)
    config = DotConfig(config)
    config.name = exp_name

    # Set file for logging
    log_filename = config.name + '.log'
    log_dir = config.log_dir_path + config.name
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    log_filename = log_dir + '/predict_log.log'
    if os.path.exists(log_filename):
        now = datetime.datetime.now()
        log_filename = log_dir + '/predict_log_' + str(now.minute) + '_' + str(now.second) + '.log'
    logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(levelname)s: %(message)s')
    logging.info(f'Configuration file used is <{config.name}>\n')

    # Check for cuda availability
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Device is {device}')

    # Build network according to config file and send it to device
    net = build_network(config)
    net.to(device=device)

    # Build dataset according to config file
    dataset = build_dataset(config)

    # Load weights
    checkpoint_path = 'checkpoints/' + config.name + '/checkpoint_best.pth'
    net.load_state_dict(torch.load(checkpoint_path, map_location=device))

    # Generate prediction
    predict(args, config, net, dataset, device)


###Train + eval

In [ ]:
# Train and evaluate
EXP_NAME = 'experiment_BEST'
run(EXP_NAME)

###Predict

In [ ]:
# Make prediction and create submission file
EXP_NAME = 'experiment_BEST'
MODEL_NAME = 'experiment_BEST'
make_prediction(EXP_NAME, MODEL_NAME, save=False)